# Otimização Natural - 2º Lista de Exercícios

Aluno: Fernando Dias

Para essa lista de exercício, foram utilizados as seguintes bibliotecas:

In [35]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Questão 1

Nessa questão, temos a matriz de transição $M$:

In [36]:
M = np.array([
    [.5,.25,.25],
    [.25,.5,.25],
    [.25,.25,.5]
])

## Item 1-a

Dado que a probabilidade em $X(0)$ é dada pela matriz $p_0=\begin{bmatrix}0.3&0.4&0.3\end{bmatrix}^T$, podemos calcular a probabilidade em $p_3$ segundo a equação: $$p_3=M^3p_0$$. Assim, temos:

In [37]:
p0 = np.array([[.3,.4,.3]]).T
resultado = M@M@M@p0 # O Operador @ representa multiplicação de matrizes
resultado

array([[0.3328125],
       [0.334375 ],
       [0.3328125]])

## Item 1-b

Dado que $X(0)=1$, podemos calcular a probabilidade de estar em qualquer outro estado em $X(3)$ da mesma forma:

In [38]:
p0 = np.array([[0,1,0]]).T
resultado = M@M@M@p0
resultado

array([[0.328125],
       [0.34375 ],
       [0.328125]])

Assim, fazemos um sorteio e definimos o estado atual:

In [39]:
valoresSorteio = np.cumsum(resultado.T) # Calcula a CDF dessas probabilidades
resultadoSorteio = np.random.uniform(0,1) # Sorteia um valor entre 0 e 1
print("X(3) =", np.argmax(resultadoSorteio < valoresSorteio)) # Com base no sorteio, diz qual o estado atual

X(3) = 1


# Item 1-c

Devemos repetir o processo acima 100 vezes e registrar as mudanças de estado em cada sorteio. Para isso, vamos definir uma matriz $P_0\in\{0,1\}^{100\times 3}$ que contém a probabilidade de estar em um estado no momento $X(0)$. Essa matriz define a presença em um estado como 1 e os outros estados como 0.

In [102]:
P0 = np.random.randint(0,2,size=(3,1000)) # Sorteando os estados iniciais aleatoriamente
P0 = P0[:,P0.sum(axis=0)==1][:,:100] # Mantendo apenas 100 amostras cujo vetor contenha apenas um elemento 1 e o resto 0
P0 = P0.T
P0.shape

(100, 3)

O próximo passo é definir como fazer uma realização do processo de markov e definir os valores da matriz $P_1$. Para isso, foi construída uma função que faz a multiplicação da matriz de transição $M$ com cada vetor-linha de uma matriz de probabilidade $P$, realiza o mesmo tipo de sorteio para cada vetor-linha e constrói a matriz de transição

In [99]:
def PassoMarkovMultiplosVetores(M,P):
    """
    Essa função pega uma matriz de transição M e uma lista de vetores P e realiza um passo na cadeia de markov definida por M.
    O resultado é um vetor de mesma dimensão de P com a probabilidade já definida como 1 no estado seguinte a P e 0 nos outros.
    """
    # Essa função faz a multiplicação de matriz para todos os 100 vetores
    ProbabilidadesDoProximoEstado = np.einsum("ji,kil->kjl",M,P)
    # Essa faz o sorteio e define o indice do estado para 
    ProximoEstado = np.argmax(np.random.uniform(0,1,size=(100,1,1)) < np.cumsum(ProbabilidadesDoProximoEstado,axis=1),axis=1)
    # Essa matriz reescreve as probabilidades do próximo passo 
    return np.eye(3)[ProximoEstado.T[0]][:,:,np.newaxis]


(100, 3, 1)

In [96]:
P0.shape

(100, 3, 1)